### Import libraries

In [1]:
import sys
import os
import gc
import numpy as np
import pandas as pd 
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")

In [2]:
home = os.path.expanduser("~")
sys.path.append(f"{home}/Documents/projects/CarPriceRegression/Machine_Learning/scripts/")

In [6]:
from util import *
%load_ext autoreload
%autoreload 2

In [7]:
set_gpu_limit(6)

### Import data for regular NN without categorical embedding

In [8]:
data = pd.read_csv(f"{home}/Documents/projects/CarPriceRegression/Processed_Data/car_onehot_data.csv")
features, y = data.drop("price",axis=1), data.price
X_train, X_test, y_train, y_test = train_test_split(features,y,test_size=0.1,random_state=2020)
X_train,X_dev,y_train,y_dev = train_test_split(X_train,y_train,test_size=0.11,random_state=2020)

### Garbage collect

In [9]:
gc.collect()
%reset -f out

Flushing output cache (0 entries)


### Set up base parameter for a base regression model

In [10]:
BATCH_SIZE = 256 
input_size = features.shape[1]
MAX_EPOCH = 1000
lr = 1e-3
l2 = 0 
SIZES = [512,512]
METRICS = ["MAPE","MAE"]

### Set up a few common parameter and make tensorflow dataset

* notice: this dataset has pretty similar scale so no reason to normalize to help with gradient descent

In [11]:
train, dev = make_tensor_dataset(X_train,y_train,256), make_tensor_dataset(X_dev,y_dev,256)

### Set up base parameter for a base regression model

In [12]:
base_model = make_model(SIZES,input_size,METRICS,l2,lr)

In [13]:
base_model.summary()

Model: "NN_regressor"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               105984    
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 513       
Total params: 369,153
Trainable params: 369,153
Non-trainable params: 0
_________________________________________________________________


### Define early stop

In [25]:
earlystop = tfk.callbacks.EarlyStopping(monitor="val_loss",mode="min",
                                        patience=10,verbose=1,restore_best_weights=True)

In [ ]:
# history = base_model.fit(train, epochs=MAX_EPOCH,shuffle=True,
#                          verbose = 1, validation_data=dev,
#                          callbacks=[earlystop])

In [ ]:
# base_model.save("base_nn_regression_mdl.h5")

In [14]:
base_model = tfk.models.load_model("base_nn_regression_mdl.h5",custom_objects={"leaky_relu":tf.nn.leaky_relu})

In [16]:
# plot_metrics(history,"loss")

In [ ]:
# plot_metrics(history,"MAPE")

### Check base model result

In [17]:
base_model.evaluate(X_test.values,y_test.values,batch_size=BATCH_SIZE)

39/39 [==============================] - 0s 1ms/step - loss: 5860600.5000 - mean_absolute_percentage_error: 10.2301 - mean_absolute_error: 1548.1078


[5860600.5, 10.230082511901855, 1548.1077880859375]

In [18]:
regression_metrics(base_model,X_train.values,y_train,X_test.values,y_test,256)

,r2_score,rmse,price_diff_abs_max
train,0.943976,2334.725640,898.469871
test,0.937015,2420.867492,1159.450571


#### Definitely can benefit from tuning. 

In [19]:
bigger_model = make_model([2056,2014,512,512],input_size,METRICS,1e-4,lr)

In [20]:
bigger_model.summary()

Model: "NN_regressor"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 2056)              425592    
_________________________________________________________________
dense_4 (Dense)              (None, 2014)              4142798   
_________________________________________________________________
dense_5 (Dense)              (None, 512)               1031680   
_________________________________________________________________
dense_6 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 513       
Total params: 5,863,239
Trainable params: 5,863,239
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# history_2 = bigger_model.fit(train, epochs=MAX_EPOCH,shuffle=True,
#                          verbose = 1, validation_data=dev,
#                          callbacks=[earlystop])

In [ ]:
# bigger_model.save("bigger_model_nn.h5")

In [21]:
bigger_model = tfk.models.load_model("bigger_model_nn.h5",custom_objects={"leaky_relu":tf.nn.leaky_relu})

In [22]:
regression_metrics(bigger_model,X_train.values,y_train,X_test.values,y_test,256)

,r2_score,rmse,price_diff_abs_max
train,0.947901,2251.459161,771.684800
test,0.938210,2397.796630,515.268254


### Pretty decent result and not over training the train data either

In [23]:
bigger_model_2 = make_model([2056,2014,512,512],input_size,METRICS,1e-4,5e-5)

In [26]:
# history_2 = bigger_model_2.fit(train, epochs=MAX_EPOCH,shuffle=True,
#                          verbose = 1, validation_data=dev,
#                          callbacks=[earlystop])

Epoch 1/1000
311/311 [==============================] - 1s 4ms/step - loss: 226725296.0000 - MAPE: 70.6021 - MAE: 11414.6680 - val_loss: 90594080.0000 - val_MAPE: 48.1973 - val_MAE: 6497.3301
Epoch 2/1000
311/311 [==============================] - 1s 4ms/step - loss: 54850036.0000 - MAPE: 34.5587 - MAE: 4802.0396 - val_loss: 36151400.0000 - val_MAPE: 21.8560 - val_MAE: 3464.3315
Epoch 3/1000
311/311 [==============================] - 1s 4ms/step - loss: 26535758.0000 - MAPE: 20.1148 - MAE: 3153.9172 - val_loss: 27528064.0000 - val_MAPE: 18.8729 - val_MAE: 3000.1492
Epoch 4/1000
311/311 [==============================] - 1s 4ms/step - loss: 20960424.0000 - MAPE: 18.0187 - MAE: 2802.3606 - val_loss: 23638520.0000 - val_MAPE: 17.3038 - val_MAE: 2751.0171
Epoch 5/1000
311/311 [==============================] - 1s 4ms/step - loss: 18115302.0000 - MAPE: 16.8210 - MAE: 2596.2922 - val_loss: 21328802.0000 - val_MAPE: 16.2859 - val_MAE: 2594.0388
Epoch 6/1000
311/311 [==========================

311/311 [==============================] - 1s 3ms/step - loss: 6982209.5000 - MAPE: 10.4048 - MAE: 1633.0812 - val_loss: 9796548.0000 - val_MAPE: 10.5434 - val_MAE: 1701.4468
Epoch 45/1000
311/311 [==============================] - 1s 4ms/step - loss: 6956488.0000 - MAPE: 10.4062 - MAE: 1633.1868 - val_loss: 9782623.0000 - val_MAPE: 10.5749 - val_MAE: 1698.1931
Epoch 46/1000
311/311 [==============================] - 1s 3ms/step - loss: 6885289.0000 - MAPE: 10.3558 - MAE: 1624.6709 - val_loss: 9712295.0000 - val_MAPE: 10.3303 - val_MAE: 1687.9277
Epoch 47/1000
311/311 [==============================] - 1s 4ms/step - loss: 6861200.0000 - MAPE: 10.3547 - MAE: 1623.9962 - val_loss: 9667131.0000 - val_MAPE: 10.6542 - val_MAE: 1702.6150
Epoch 48/1000
311/311 [==============================] - 1s 4ms/step - loss: 6797789.0000 - MAPE: 10.3225 - MAE: 1618.8098 - val_loss: 9702628.0000 - val_MAPE: 11.1235 - val_MAE: 1732.8860
Epoch 49/1000
311/311 [==============================] - 1s 3ms/step 

311/311 [==============================] - 1s 4ms/step - loss: 5669545.0000 - MAPE: 9.8731 - MAE: 1531.6514 - val_loss: 8560518.0000 - val_MAPE: 10.0808 - val_MAE: 1626.4806
Epoch 88/1000
311/311 [==============================] - 1s 3ms/step - loss: 5663000.5000 - MAPE: 9.8809 - MAE: 1532.9290 - val_loss: 8571597.0000 - val_MAPE: 9.9412 - val_MAE: 1625.0104
Epoch 89/1000
311/311 [==============================] - 1s 4ms/step - loss: 5654260.0000 - MAPE: 9.8784 - MAE: 1532.2981 - val_loss: 8519572.0000 - val_MAPE: 9.9619 - val_MAE: 1617.5585
Epoch 90/1000
311/311 [==============================] - 1s 4ms/step - loss: 5626422.0000 - MAPE: 9.8691 - MAE: 1529.1700 - val_loss: 8476436.0000 - val_MAPE: 10.0190 - val_MAE: 1615.8660
Epoch 91/1000
311/311 [==============================] - 1s 4ms/step - loss: 5602161.5000 - MAPE: 9.8434 - MAE: 1527.0750 - val_loss: 8514585.0000 - val_MAPE: 9.9262 - val_MAE: 1616.6920
Epoch 92/1000
311/311 [==============================] - 1s 4ms/step - loss: 

311/311 [==============================] - 1s 3ms/step - loss: 5224421.0000 - MAPE: 9.6128 - MAE: 1487.6044 - val_loss: 8187854.0000 - val_MAPE: 9.7970 - val_MAE: 1602.6128
Epoch 131/1000
311/311 [==============================] - 1s 4ms/step - loss: 5205700.0000 - MAPE: 9.6089 - MAE: 1485.3898 - val_loss: 8094090.5000 - val_MAPE: 10.3785 - val_MAE: 1623.1178
Epoch 132/1000
311/311 [==============================] - 1s 4ms/step - loss: 5208399.5000 - MAPE: 9.6249 - MAE: 1488.0859 - val_loss: 8048458.0000 - val_MAPE: 9.8689 - val_MAE: 1587.5741
Epoch 133/1000
311/311 [==============================] - 1s 4ms/step - loss: 5194735.5000 - MAPE: 9.5966 - MAE: 1483.7816 - val_loss: 8128985.0000 - val_MAPE: 9.7460 - val_MAE: 1595.3971
Epoch 134/1000
311/311 [==============================] - 1s 3ms/step - loss: 5196992.5000 - MAPE: 9.6084 - MAE: 1485.9644 - val_loss: 8072468.5000 - val_MAPE: 9.7628 - val_MAE: 1588.1193
Epoch 135/1000
311/311 [==============================] - 1s 4ms/step - lo

In [27]:
# bigger_model_2.save("bigger_model_2.h5")

In [ ]:
bigger_model_2 = tfk.models.load_model("bigger_model_2.h5",custom_objects={"leaky_relu":tf.nn.leaky_relu})

In [28]:
regression_metrics(bigger_model_2,X_train.values,y_train,X_test.values,y_test,256)

,r2_score,rmse,price_diff_abs_max
train,0.948620,2235.873711,816.022391
test,0.939284,2376.859028,582.943434


### Looking at cases where price_diff_abs_max is high

In [29]:
price_table = price_diff(bigger_model,features,y,batch_size=256)

In [32]:
price_table.sort_values("price_diff_abs",ascending=False).head(20)

,binned_year,mil_cat,binned_mpg,engine_binned,model_ 2 Series,model_ 3 Series,model_ 4 Series,model_ 5 Series,model_ 6 Series,model_ 7 Series,...,model_EcoSport,transmission_Manual,transmission_Other,transmission_Semi-Auto,fuelType_Electric,fuelType_Hybrid,fuelType_Other,fuelType_Petrol,price_diff_pct,price_diff_abs
11984,1,0,3,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,-771.684800,771.684800
39711,1,2,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-722.626006,722.626006
71819,1,0,2,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,-593.055130,593.055130
39978,1,5,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,-515.268254,515.268254
20061,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,-422.928905,422.928905
47201,1,5,2,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,-409.967556,409.967556
69135,1,4,2,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,-407.544633,407.544633
44539,1,4,3,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,-391.214094,391.214094
39540,1,5,1,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,-350.905927,350.905927
40171,1,4,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,-278.709283,278.709283


### Most of these models are older. The top cases 